In [2]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine

# Informations de connexion à la base de données
host = 'postgres_dst' 
port = '5432'
database = 'dst_airlines_db'
user = 'dst_reader'
password = 'pass_reader'

# Connexion à la base de données
try:
    engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}')
    # Charger les données de la table dans un DataFrame pandas
    query = 'SELECT * FROM l2.operations_customer_flight_info'  # Requête SQL explicite avec schéma et table
    df = pd.read_sql_query(query, con=engine)
    print("Connexion réussie et données chargées dans le DataFrame !")
except Exception as e:
    print("Erreur lors de la connexion ou du chargement des données :", e)

# Afficher les premières lignes du DataFrame pour vérifier
df.head()

Connexion réussie et données chargées dans le DataFrame !


,id,departure_airport_code,departure_scheduled_datetime,departure_estimated_datetime,departure_actual_datetime,departure_terminal_name,departure_terminal_gate,departure_status_code,departure_status_description,arrival_airport_code,...,arrival_actual_datetime,arrival_terminal_name,arrival_terminal_gate,arrival_status_code,arrival_status_description,operating_airline_id,operating_flight_nb,equipment_aircraft_code,overall_status_code,overall_status_description
0,95011,BRU,2024-07-19 10:50:00,NaT,2024-07-19 10:59:00,None,B33,DP,Flight Departed,LHR,...,2024-07-19 10:25:00,None,None,LD,Flight Landed,SN,2093,320,LD,Flight Landed
1,95012,DUS,2024-07-19 07:50:00,NaT,2024-07-19 08:06:00,None,A49,DP,Flight Departed,ATH,...,2024-07-19 12:57:00,None,None,LD,Flight Landed,EW,9680,32N,LD,Flight Landed
2,93703,CDG,2024-07-06 10:45:00,NaT,NaT,1,6068,NO,No Status,ZRH,...,NaT,None,None,NO,No Status,2L,633,E90,CD,Flight Cancelled
3,93704,ZRH,2024-07-06 18:35:00,NaT,NaT,None,A65,NO,No Status,AMS,...,NaT,None,None,NO,No Status,LX,734,223,CD,Flight Cancelled
4,93705,AMS,2024-07-06 21:05:00,NaT,NaT,None,None,NO,No Status,ZRH,...,NaT,None,None,NO,No Status,LX,735,223,CD,Flight Cancelled
